# HeAT Tutorial

Inspired by the [CS228 tutorial](https://github.com/kuleshov/cs228-material/blob/master/tutorials/python/cs228-python-tutorial.ipynb) by Volodomyr Kuleshov and Isaac Caswell.

## Introduction

**Table of Contents**

* [Installation](#Installation)
    * [Dependencies](#Dependencies)
    * [Dependencies](#Dependencies)
* [HeAT Arrays](#HeAT-Arrays)
    * [Datatypes](#Datatypes)
    * [Operations](#Operations)
    * [Indexing](#Indexing)
* [Parallel Processing](#Parallel-Processing)
    * [GPUs](#Dependencies)
    * [Distributed Computing](#Distributed-Computing)
    * [Dos and Don'ts](#Dos-and-Don'ts)
    

HeAT is a flexible and seamless open-source software for high performance data analytics and machine learnings. It provides highly optimized algorithms and data structures for tensor computations using CPUs, GPUs and distributed cluster systems on top of MPI. The goal of HeAT is to fill the gap between data analytics and machine learning libraries with a strong focus on on single-node performance, and traditional high-performance computing (HPC). HeAT's generic Python-first programming interface integrates seamlessly with the existing data science ecosystem and makes it as effortless as using numpy to write scalable scientific and data science applications.

HeAT allows you tackle your actual Big Data challenges that go beyond the computational and memory needs of your laptop and desktop.

HeAT provides a high-performance, distributed multidimensional array object, algorithms and tools for working with these arrays. Targeting 

For this tutorial, we assume that you are somewhat proficient in the Python programming language. Equally, it is beneficial that you have worked with vectorized multi-dimensional array data structures before, as offered by NumPy, Matlab or R for example. If not or you feel like refreshing your knowledge, you might find the following ressources useful: [CS228 Python and NumPy Tutorial](https://github.com/kuleshov/cs228-material/blob/master/tutorials/python/cs228-python-tutorial.ipynb), [NumPy for MATLAB users](https://docs.scipy.org/doc/numpy/user/numpy-for-matlab-users.html) and [NumPy for R users](http://mathesaurus.sourceforge.net/r-numpy.html)

In line with this tutorial, we will cover the following topics

* Installation and setup of HeAT
* Working with HeAT arrays, operations, indexing etc.
* Utilizing HeAT's scalable parallel processing capabilities

## Installation

### Dependencies

### Optional Features

## HeAT Arrays

### Datatypes

### Operations

### Indexing

## Parallel Processing

### GPUs

### Distributed Computing

### Dos and Don'ts